# Whiten the data

In [ ]:
import numpy as np
import scipy.fft
import matplotlib.pyplot as plt

from gwpy.timeseries import TimeSeries
from gwpy.frequencyseries import FrequencySeries

In [ ]:
suffixes = ['27hrs', 'event_120s', 'event_4096s']  # suffixes for channels

channels = ['DCS-CALIB_STRAIN_CLEAN_C01_512Hz', 'LSC-POP_A_RF45_I_ERR_DQ_512Hz',
           'LSC-POP_A_RF45_Q_ERR_DQ_512Hz', 'LSC-POP_A_RF9_I_ERR_DQ_512Hz']

In [ ]:
for suffix in suffixes:
    for channel in channels:

        # load ASD
        asd = FrequencySeries.read(f'data/{channel}_ASD.txt')

        # load tseries
        tseries = np.load(f'data/{channel}_{suffix}.npy')[:,1]
        gps_times = np.load(f'data/{channel}_{suffix}.npy')[:,0]

        # tseries in freq domain
        fseries = scipy.fft.rfft(tseries)

        # interpolate ASD to fseries length
        freqs = asd.frequencies
        freqs_interp = np.linspace(freqs[0], freqs[-1], len(fseries))
        asd_interp = np.interp(freqs_interp, freqs, asd.value)

        # whiten tseries
        whitened = scipy.fft.irfft(fseries/asd_interp)

        # write whitened tseries to a np file
        whitened_array = np.array([gps_times, whitened]).T
        np.save(f'real_data/{channel}_{suffix}_whitened', whitened_array)